In [2]:
import pandas as pd
import re



In [3]:
df_hb = pd.read_csv('a:/bloeding-met-patientenlijst-gedetailleerd/bloeding-met-patientenlijst-10-lab.csv')
df_hb['lab_measurement_start_date'] = pd.to_datetime(df_hb['lab_measurement_start_date'])

df_hb

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Bloedgroep,A neg,NaN,NaN,2024-08-05 00:00:00
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Kreatinine,83,83.0,µmol/L,2024-07-31 09:11:00
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,eGFR CKD-EPI,58,58.0,ml/min/{1.73_m2},2024-07-31 09:11:00
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,NT-proBNP,780,780.0,pg/ml,2024-07-22 09:29:00
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.9,6.9,mmol/L,2024-07-22 09:29:00
...,...,...,...,...,...,...
10428,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Alkalische Fosfatase (AF),88,88.0,U/L,2018-10-17 11:11:00
10429,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,GGT,48,48.0,U/L,2018-10-17 11:11:00
10430,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,ASAT,25,25.0,U/L,2018-10-17 11:11:00
10431,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,ALAT,19,19.0,U/L,2018-10-17 11:11:00


In [4]:
df_hb['lab_measurement_description'].unique()

array(['Bloedgroep', 'Kreatinine', 'eGFR CKD-EPI', 'NT-proBNP',
       'Hemoglobine', 'MCV', 'Trombocyten', 'Leucocyten', 'Natrium',
       'Kalium', 'Alkalische Fosfatase (AF)', 'CRP', 'TF/log FERR',
       'Neutrofielen', 'Lymfocyten', 'Monocyten', 'Eosinofielen',
       'Calcium', 'Magnesium', 'Ureum', 'GGT', 'ASAT', 'ALAT',
       'Actief B12', 'Foliumzuur', 'IJzer', 'Transferrine',
       'Transferrineverzadiging', 'Ferritine', 'Albumine', 'Vit.D3 25 OH',
       'TSH(screen)', 'Calcium corr', 'Informatie', 'Neutrofielen #',
       'Lymfocyten #', 'Monocyten #', 'Basofielen #', 'INR',
       'Nitriet (ur)', 'Plaveisel epitheel (kwantitatief)', 'Bacteriën',
       'Info urine', 'Materiaal :', 'Klin./Medische gegevens',
       'Antibiotica', 'Extra onderzoek op', 'Urine kweek', 'Leukocyten',
       'Erytrocyten', 'Extra buis', 'Hematocriet', 'Glucose',
       'Microalbumine(u)', 'Alb/Kreat R', 'Fosfaat', 'HbA1c',
       'Bilirubine Totaal', 'LD', 'Eosinofielen #', 'PSA', 'Opm. afname

In [34]:
# Define threshold for significant drop
SIGNIFICANT_DROP_THRESHOLD = 6.85
df_hb['hb_pct_change']  = None
df_hb['significant_hb_drop'] = None

# Sort by patient and time
# df_hb = df_hb.sort_values(by=['pseudo_id', 'lab_measurement_start_date', 'lab_measurement_description'])
df_hb = df_hb.sort_values(by=['pseudo_id', 'lab_measurement_description', 'lab_measurement_start_date'])

for i in range(len(df_hb)):
    if df_hb['pseudo_id'].iloc[i] == df_hb['pseudo_id'].iloc[i-1]:
        # Check if the current and previous rows are within 7 days
        time_diff = (df_hb['lab_measurement_start_date'].iloc[i] - df_hb['lab_measurement_start_date'].iloc[i-1]).days
        if df_hb['lab_measurement_description'].iloc[i] == 'Hemoglobine' and df_hb['lab_measurement_description'].iloc[i-1] == 'Hemoglobine':
            
            # Calculate % change in Hb per patient
            # df_hb['hb_pct_change'].iloc[i] = (df_hb['lab_measurement_test_value_numeric'].iloc[i] - df_hb['lab_measurement_test_value_numeric'].iloc[i-1]).pct_change(periods=-1) * -100
            percentage_channge = ((df_hb['lab_measurement_test_value_numeric'].iloc[i] - df_hb['lab_measurement_test_value_numeric'].iloc[i-1]) / df_hb['lab_measurement_test_value_numeric'].iloc[i-1]) * -100
            print(f"Percentage change for patient {df_hb['pseudo_id'].iloc[i]}: {percentage_channge:.2f}%")
            df_hb.at[i, 'hb_pct_change'] = percentage_channge
    # if df_hb['lab_measurement_description'].iloc[i] == 'Hemoglobine':
            
    #         # Calculate % change in Hb per patient
    #         df_hb['hb_pct_change'].iloc[i] = df_hb.groupby('pseudo_id')['lab_measurement_test_value_numeric'].pct_change(periods=-1) * -100  # negative drop

    #         # Flag significant drops (e.g. ≥ 6.85%) within 7 days
    #         df_hb['significant_hb_drop'] = df_hb['hb_pct_change'] >= SIGNIFICANT_DROP_THRESHOLD
    # else:
    #     # If not 'Hemoglobine', set to NaN
    #     df_hb['hb_pct_change'] = None
    #     df_hb['significant_hb_drop'] = None

df_hb

Percentage change for patient 046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6: -11.29%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: -1.08%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: 10.64%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: 4.76%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: 1.25%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: -16.46%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: 1.09%
Percentage change for patient 088C9FD98B8B2CBCB597C17C07AC1845B21F0849: 8.79%
Percentage change for patient 0A5645E02FA818D1629926B6BEFA81CF91C25A46: 1.61%
Percentage change for patient 0A5645E02FA818D1629926B6BEFA81CF91C25A46: 1.64%
Percentage change for patient 0A5645E02FA818D1629926B6BEFA81CF91C25A46: -6.67%
Percentage change for patient 0BC512A65442D0BB4B00FBE05E7EA6283E5C11FD: -5.36%
Percentage change for patient 0BC512A65442D0BB4B00FBE05E

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,hb_pct_change,significant_hb_drop
33,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,ALAT,14,14.00,U/L,2024-07-15 08:38:00,None,None
32,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,ASAT,19,19.00,U/L,2024-07-15 08:38:00,None,None
35,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Actief B12,570,570.00,pmol/L,2024-07-15 08:38:00,None,None
41,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Albumine,22,22.00,g/L,2024-07-15 08:38:00,None,None
30,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Alkalische Fosfatase (AF),106,106.00,U/L,2024-07-15 08:38:00,None,None
...,...,...,...,...,...,...,...,...
10361,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,hs Troponine T,38,38.00,ng/l,2021-03-09 12:25:00,None,None
10356,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,hs Troponine T,35,35.00,ng/l,2021-03-09 14:33:00,None,None
10407,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,pCO2,39,39.00,mm Hg,2021-03-01 14:50:00,None,None
10408,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,pH,7.53,7.53,NaN,2021-03-01 14:50:00,None,None


In [35]:
# df_hb = df_hb.sort_values(by=['pseudo_id', 'lab_measurement_description', 'lab_measurement_start_date'])
# df_hb

In [41]:
relevant_tests = [
    'Hemoglobine',
    # 'Hematocriet',
    # 'INR',
    # 'APTT',
    # 'Protrombinetijd',
    # 'Ureum',
    # 'MCV',
    # 'Trombocyten'
]

df_hb_filtered = df_hb[df_hb['lab_measurement_description'].isin(relevant_tests)]
df_hb_filtered


,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,hb_pct_change,significant_hb_drop
16,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.2,6.2,mmol/L,2024-07-15 08:38:00,None,None
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.9,6.9,mmol/L,2024-07-22 09:29:00,None,None
192,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.3,9.3,mmol/L,2024-06-17 11:40:00,None,None
186,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.4,9.4,mmol/L,2024-06-17 11:41:00,None,None
164,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.4,8.4,mmol/L,2024-08-13 13:36:00,None,None
...,...,...,...,...,...,...,...,...
10237,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,6.5,6.5,mmol/L,2020-06-22 08:00:00,None,None
10417,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.3,7.3,mmol/L,2018-10-17 11:11:00,None,None
10386,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.2,7.2,mmol/L,2021-03-01 14:50:00,None,None
10357,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,8.7,8.7,mmol/L,2021-03-09 12:25:00,None,None


In [43]:
print(len(df_hb_filtered[df_hb_filtered['hb_pct_change'] > 0]))
df_hb_filtered['hb_pct_change'].value_counts()

18


-7.317073     2
 4.761905     1
-6.666667     1
-16.071429    1
 9.523810     1
-0.000000     1
-1.408451     1
 3.846154     1
 4.597701     1
-10.958904    1
-7.894737     1
 29.032258    1
 24.390244    1
-36.666667    1
 7.500000     1
 20.987654    1
-20.895522    1
 1.250000     1
 62.385321    1
 4.615385     1
 10.769231    1
-14.814815    1
 4.301075     1
-2.298851     1
-61.904762    1
 14.516129    1
-3.703704     1
 6.896552     1
-27.500000    1
-3.614458     1
-3.174603     1
-18.000000    1
-36.000000    1
-5.357143     1
-3.409091     1
-2.439024     1
-20.000000    1
 9.722222     1
-5.882353     1
 4.000000     1
-47.692308    1
-7.142857     1
-5.813953     1
 8.333333     1
Name: hb_pct_change, dtype: int64

In [39]:
# Define threshold for significant drop
SIGNIFICANT_DROP_PCT = 6.85

# Sort by patient and time
df_hb_filtered = df_hb_filtered.sort_values(by=['pseudo_id', 'lab_measurement_start_date'])

# Calculate time difference to the next Hb measurement (in days)
df_hb_filtered['timestamp_next'] = df_hb_filtered.groupby('pseudo_id')['lab_measurement_start_date'].shift(-1)
df_hb_filtered['days_to_next'] = (df_hb_filtered['timestamp_next'] - df_hb_filtered['lab_measurement_start_date']).dt.days

# Filter only those within 7 days
df_hb_filtered['within_7_days'] = df_hb_filtered['days_to_next'] <= 7

# Calculate % change in Hb per patient
df_hb_filtered['hb_pct_change'] = df_hb_filtered.groupby('pseudo_id')['lab_measurement_test_value_numeric'].pct_change(periods=-1) * -100  # negative drop

# Flag significant drops (e.g. ≥ 6.85%) within 7 days
SIGNIFICANT_DROP_THRESHOLD = 6.85
df_hb_filtered['significant_hb_drop'] = (
    df_hb_filtered['hb_pct_change'] >= SIGNIFICANT_DROP_THRESHOLD
) & df_hb_filtered['within_7_days']

df_hb_filtered


,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,significant_hb_drop,hb_pct_change,timestamp_next,days_to_next,within_7_days
16,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.2,6.2,mmol/L,2024-07-15 08:38:00,True,10.144928,2024-07-22 09:29:00,7.0,True
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.9,6.9,mmol/L,2024-07-22 09:29:00,False,NaN,NaT,NaN,False
192,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.3,9.3,mmol/L,2024-06-17 11:40:00,False,1.063830,2024-06-17 11:41:00,0.0,True
186,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.4,9.4,mmol/L,2024-06-17 11:41:00,False,-11.904762,2024-08-13 13:36:00,57.0,False
164,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.4,8.4,mmol/L,2024-08-13 13:36:00,False,-5.000000,2024-10-25 13:06:00,72.0,False
...,...,...,...,...,...,...,...,...,...,...,...
10237,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,6.5,6.5,mmol/L,2020-06-22 08:00:00,False,NaN,NaT,NaN,False
10417,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.3,7.3,mmol/L,2018-10-17 11:11:00,False,-1.388889,2021-03-01 14:50:00,866.0,False
10386,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.2,7.2,mmol/L,2021-03-01 14:50:00,True,17.241379,2021-03-09 12:25:00,7.0,True
10357,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,8.7,8.7,mmol/L,2021-03-09 12:25:00,False,2.247191,2021-03-10 09:30:00,0.0,True


In [40]:
# Filter rows with significant Hb drops
significant_drops = df_hb_filtered[df_hb_filtered['significant_hb_drop']]
significant_drops

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,significant_hb_drop,hb_pct_change,timestamp_next,days_to_next,within_7_days
16,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.2,6.2,mmol/L,2024-07-15 08:38:00,True,10.144928,2024-07-22 09:29:00,7.0,True
765,12D65DA1F4CFAC101DE53C050C9037D97F42FC18,Hemoglobine,5.1,5.1,mmol/L,2022-07-05 08:00:00,True,10.526316,2022-07-06 08:00:00,1.0,True
761,12D65DA1F4CFAC101DE53C050C9037D97F42FC18,Hemoglobine,5.7,5.7,mmol/L,2022-07-06 08:00:00,True,9.523810,2022-07-07 08:00:00,1.0,True
720,12D65DA1F4CFAC101DE53C050C9037D97F42FC18,Hemoglobine,4.6,4.6,mmol/L,2022-07-19 11:50:00,True,14.814815,2022-07-21 15:44:00,2.0,True
897,15CF926899FB0141DABA60251D292FCC89C94C1B,Hemoglobine,4.4,4.4,mmol/L,2022-05-19 13:46:00,True,24.137931,2022-05-20 08:00:00,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...
10011,F797097BA39F81B05EEC3F251973743CA03CB9B4,Hemoglobine,5.1,5.1,mmol/L,2024-12-23 20:00:00,True,17.741935,2024-12-24 02:42:00,0.0,True
10005,F797097BA39F81B05EEC3F251973743CA03CB9B4,Hemoglobine,5.5,5.5,mmol/L,2024-12-24 08:00:00,True,16.666667,2024-12-24 20:00:00,0.0,True
9984,F797097BA39F81B05EEC3F251973743CA03CB9B4,Hemoglobine,5.8,5.8,mmol/L,2024-12-25 14:30:00,True,13.432836,2024-12-26 08:00:00,0.0,True
10260,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.6,5.6,mmol/L,2020-06-19 11:00:00,True,13.846154,2020-06-22 08:00:00,2.0,True


In [41]:
hb_patient_lst_7_days = significant_drops['pseudo_id'].unique().tolist()
print(len(hb_patient_lst_7_days))
hb_patient_lst_7_days

47


['046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6',
 '12D65DA1F4CFAC101DE53C050C9037D97F42FC18',
 '15CF926899FB0141DABA60251D292FCC89C94C1B',
 '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682',
 '29B3653AE690547AB14AC7FCD32B21A561D5FF9A',
 '2E7E9399C8366C94770E35BC822203C1B0BDAB07',
 '355F07B9B6AF154431F346CBC4A6722294250C43',
 '3605432FE03B28514E3927DA7E5C53BD177BFD31',
 '37099C38CFE1055CF6950B3D61CEC774849364D9',
 '37BFA228DFA04BE0596DCCC63D245B42B3A15727',
 '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE',
 '40FF2EE78841C453C1C673E9047BABC03DD6CCEE',
 '4D22E67480A038F4FEA67046F5706560CBB06C27',
 '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C',
 '618F149E27BFA64EBFEC903D49A75AAF88BD75B2',
 '67A836EEF5B75637507621800CA8F8C0CAC23DCE',
 '6CF1734B040468B6802E26DFA5BC6B6FF6869362',
 '706C18D256797B3BAFC980C49CA839BB467F6B5F',
 '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B',
 '708E355575D017D3349A8FADE56EC4CE1059C187',
 '7561B5E32DE738229F2D658FBAACEFFE80B50600',
 '895342BFD0D1C32FFF8F009C0E20CBE97B04677D',
 '89AC6D36

Taking treatments/diagnosis into a count

In [42]:
df_dbc = pd.read_csv('a:/bloeding-met-patientenlijst-gedetailleerd/bloeding-met-patientenlijst-2-diagnose-db-cs.csv')
df_dbc['diagnose_db_cs_dbc_start_date'] = pd.to_datetime(df_dbc['diagnose_db_cs_dbc_start_date'])
df_dbc['diagnose_db_cs_dbc_end_date'] = pd.to_datetime(df_dbc['diagnose_db_cs_dbc_end_date'])
df_dbc.head()

,pseudo_id,diagnose_db_cs_dbc_specialty_code,diagnose_db_cs_dbc_specialty_code_description,diagnose_db_cs_dbc_diagnosis_code,diagnose_db_cs_dbc_diagnosis_code_description,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2021-09-05,2022-01-02
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2021-05-08,2021-09-04
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2021-01-08,2021-05-07
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-11-24,2021-01-07
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",612,(acute)Bloeding distale tractus digestivus,2017-06-02,2017-09-29


In [43]:
# df = df_dbc.copy()

In [44]:
# for i in range(len(df)):
#     for j in range(len(df_hb_filtered)):
#         if df['pseudo_id'][i] == df_hb_filtered['pseudo_id'][j]:
#             date = df_hb_filtered['date'][j]

#             if df_dbc['diagnose_db_cs_dbc_start_date'][i] <= date <= df_dbc['diagnose_db_cs_dbc_end_date'][i]:
#                 df.at[i, 'lab_measurement_description'] = df_hb_filtered['lab_measurement_description'][j]
#                 df.at[i, 'lab_measurement_test_value_numeric'] = df_dbc['lab_measurement_test_value_numeric'][j]
#                 df.at[i, 'lab_measurement_start_date'] = df_hb_filtered['lab_measurement_start_date'][j]
#                 # df.at[i, 'diagnose_db_cs_dbc_end_date'] = df_dbc['diagnose_db_cs_dbc_end_date'][j]

# df

In [45]:
df_dbc['diagnosis_window_start'] = df_dbc['diagnose_db_cs_dbc_start_date'] - pd.Timedelta(days=90)
df_dbc['diagnosis_window_end'] = df_dbc['diagnose_db_cs_dbc_end_date'] + pd.Timedelta(days=7)

# Repeat Hb data for possible merging
df_merged = df_hb_filtered.merge(df_dbc, on='pseudo_id', how='inner')

# Keep only those where Hb test is within the diagnosis window
df_merged = df_merged[
    (df_merged['lab_measurement_start_date'] >= df_merged['diagnosis_window_start']) &
    (df_merged['lab_measurement_start_date'] <= df_merged['diagnosis_window_end'])
]

df_merged = df_merged[
    (df_merged['lab_measurement_start_date'] >= df_merged['diagnose_db_cs_dbc_start_date']) &
    (df_merged['lab_measurement_start_date'] <= df_merged['diagnose_db_cs_dbc_end_date'])
]

df_merged

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,significant_hb_drop,hb_pct_change,timestamp_next,days_to_next,within_7_days,diagnose_db_cs_dbc_specialty_code,diagnose_db_cs_dbc_specialty_code_description,diagnose_db_cs_dbc_diagnosis_code,diagnose_db_cs_dbc_diagnosis_code_description,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,diagnosis_window_start,diagnosis_window_end
19,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.3,9.3,mmol/L,2024-06-17 11:40:00,False,1.063830,2024-06-17 11:41:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-02-27,2024-06-25,2023-11-29,2024-07-02
32,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.4,9.4,mmol/L,2024-06-17 11:41:00,False,-11.904762,2024-08-13 13:36:00,57.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-02-27,2024-06-25,2023-11-29,2024-07-02
44,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.4,8.4,mmol/L,2024-08-13 13:36:00,False,-5.000000,2024-10-25 13:06:00,72.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-06-26,2024-10-23,2024-03-28,2024-10-30
56,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.0,8.0,mmol/L,2024-10-25 13:06:00,False,-1.265823,2024-10-25 13:07:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-10-24,2025-02-20,2024-07-26,2025-02-27
69,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,7.9,7.9,mmol/L,2024-10-25 13:07:00,False,14.130435,2025-01-29 11:07:00,95.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-10-24,2025-02-20,2024-07-26,2025-02-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.8,5.8,mmol/L,2020-06-15 09:25:00,False,-1.754386,2020-06-17 11:50:00,2.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19
5094,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.7,5.7,mmol/L,2020-06-17 11:50:00,False,-1.785714,2020-06-19 11:00:00,1.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19
5098,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.6,5.6,mmol/L,2020-06-19 11:00:00,True,13.846154,2020-06-22 08:00:00,2.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19
5102,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,6.5,6.5,mmol/L,2020-06-22 08:00:00,False,NaN,NaT,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19


In [46]:
# Sort by patient and time
df_merged = df_merged.sort_values(['pseudo_id', 'diagnose_db_cs_dbc_start_date', 'lab_measurement_start_date'])

# Group per patient AND diagnosis period
group_cols = ['pseudo_id', 'diagnose_db_cs_dbc_start_date', 'diagnose_db_cs_dbc_end_date']

# Time difference to next test
df_merged['next_test_time'] = df_merged.groupby(group_cols)['lab_measurement_start_date'].shift(-1)
df_merged['days_to_next'] = (df_merged['next_test_time'] - df_merged['lab_measurement_start_date']).dt.days

# Percent change in Hb
df_merged['hb_pct_change'] = (
    df_merged.groupby(group_cols)['lab_measurement_test_value_numeric']
    .pct_change(periods=-1) * -100
)

# Flag significant drops within 7 days
SIGNIFICANT_DROP_THRESHOLD = 6.85
df_merged['significant_hb_drop'] = (
    (df_merged['hb_pct_change'] >= SIGNIFICANT_DROP_THRESHOLD) &
    (df_merged['days_to_next'] <= 7)
)

In [47]:
# Filter only significant drops
significant_drops = df_merged[df_merged['significant_hb_drop']]

# Optional: one row per patient + diagnosis period
patients_with_drop = significant_drops[group_cols].drop_duplicates()
len(patients_with_drop)

hb_patient_lst_7_days_diagnosis = significant_drops['pseudo_id'].unique().tolist()
print(len(hb_patient_lst_7_days_diagnosis)) # 31 
hb_patient_lst_7_days_diagnosis


29


['12D65DA1F4CFAC101DE53C050C9037D97F42FC18',
 '15CF926899FB0141DABA60251D292FCC89C94C1B',
 '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682',
 '29B3653AE690547AB14AC7FCD32B21A561D5FF9A',
 '2E7E9399C8366C94770E35BC822203C1B0BDAB07',
 '355F07B9B6AF154431F346CBC4A6722294250C43',
 '3605432FE03B28514E3927DA7E5C53BD177BFD31',
 '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE',
 '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C',
 '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B',
 '708E355575D017D3349A8FADE56EC4CE1059C187',
 '7561B5E32DE738229F2D658FBAACEFFE80B50600',
 '895342BFD0D1C32FFF8F009C0E20CBE97B04677D',
 '94876C5A2D72759210A854698D5A5E012C2ED946',
 '96764B9E95688ACB64A6C8E99AC2E6D5828BED12',
 '9763CE55BAB3BF537B9309F9A7A6085F86ED878A',
 '9A5F28B3534569F4F2EF8F9C7CCD12F102A511D3',
 'A75FEC66C73E482282B5537091BFBD0814458EAA',
 'A9A2482C557733BFC15A7DEAC217F8766D37FAA1',
 'BAE82DBED446134D3759C32B5457D88A06F8F6AD',
 'C5EE7D6FA25DF773D7EF38E213CDAAA1419C1A7A',
 'C775682FCB29A890D7B64B211F17DFF3D8867951',
 'CEA4E6D3

In [48]:
patients_in_7_days_not_in_diagnosis = list(set(hb_patient_lst_7_days) - set(hb_patient_lst_7_days_diagnosis))
# patients_in_7_days_not_in_diagnosis = list(set(hb_patient_lst_7_days_diagnosis) - set(hb_patient_lst_7_days))
patients_in_7_days_not_in_diagnosis
df_merged[df_merged['pseudo_id'].isin(patients_in_7_days_not_in_diagnosis)]

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,significant_hb_drop,hb_pct_change,timestamp_next,days_to_next,within_7_days,diagnose_db_cs_dbc_specialty_code,diagnose_db_cs_dbc_specialty_code_description,diagnose_db_cs_dbc_diagnosis_code,diagnose_db_cs_dbc_diagnosis_code_description,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,diagnosis_window_start,diagnosis_window_end,next_test_time
1522,67A836EEF5B75637507621800CA8F8C0CAC23DCE,Hemoglobine,5.7,5.7,mmol/L,2021-01-29 09:30:00,False,3.389831,2021-01-30 08:00:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-10-24,2021-02-20,2020-07-26,2021-02-27,2021-01-30 08:00:00
1528,67A836EEF5B75637507621800CA8F8C0CAC23DCE,Hemoglobine,5.9,5.9,mmol/L,2021-01-30 08:00:00,False,NaN,2021-03-15 13:31:00,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-10-24,2021-02-20,2020-07-26,2021-02-27,NaT
1560,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,5.8,5.8,mmol/L,2023-06-02 12:06:00,False,4.918033,2023-07-06 10:10:00,33.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-04-07,2023-08-04,2023-01-07,2023-08-11,2023-07-06 10:10:00
1579,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,6.1,6.1,mmol/L,2023-07-06 10:10:00,False,-19.607843,2023-07-29 19:22:00,23.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-04-07,2023-08-04,2023-01-07,2023-08-11,2023-07-29 19:22:00
1598,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,5.1,5.1,mmol/L,2023-07-29 19:22:00,False,-21.428571,2023-08-03 08:20:00,4.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-04-07,2023-08-04,2023-01-07,2023-08-11,2023-08-03 08:20:00
1617,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,4.2,4.2,mmol/L,2023-08-03 08:20:00,False,NaN,2023-08-10 10:35:00,NaN,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-04-07,2023-08-04,2023-01-07,2023-08-11,NaT
1635,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,6.8,6.8,mmol/L,2023-08-10 10:35:00,False,-4.615385,2023-08-24 09:40:00,13.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-08-05,2023-12-02,2023-05-07,2023-12-09,2023-08-24 09:40:00
1654,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,6.5,6.5,mmol/L,2023-08-24 09:40:00,False,-0.000000,2023-09-18 12:05:00,25.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-08-05,2023-12-02,2023-05-07,2023-12-09,2023-09-18 12:05:00
1673,6CF1734B040468B6802E26DFA5BC6B6FF6869362,Hemoglobine,6.5,6.5,mmol/L,2023-09-18 12:05:00,False,NaN,2023-12-04 10:55:00,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2023-08-05,2023-12-02,2023-05-07,2023-12-09,NaT
1752,706C18D256797B3BAFC980C49CA839BB467F6B5F,Hemoglobine,6.4,6.4,mmol/L,2019-03-29 22:43:00,False,-8.474576,2019-03-30 08:00:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2019-03-29,2019-05-12,2018-12-29,2019-05-19,2019-03-30 08:00:00


Without 7 days rescrition

In [49]:
# Sort by patient and time
df_merged = df_merged.sort_values(['pseudo_id', 'diagnose_db_cs_dbc_start_date', 'lab_measurement_start_date'])

# Group per patient AND diagnosis period
group_cols = ['pseudo_id', 'diagnose_db_cs_dbc_start_date', 'diagnose_db_cs_dbc_end_date']

# Time difference to next test
df_merged['next_test_time'] = df_merged.groupby(group_cols)['lab_measurement_start_date'].shift(-1)
df_merged['days_to_next'] = (df_merged['next_test_time'] - df_merged['lab_measurement_start_date']).dt.days

# Percent change in Hb
df_merged['hb_pct_change'] = (
    df_merged.groupby(group_cols)['lab_measurement_test_value_numeric']
    .pct_change(periods=-1) * -100
)

# Flag significant drops within 7 days
SIGNIFICANT_DROP_THRESHOLD = 6.85
df_merged['significant_hb_drop'] = (
    (df_merged['hb_pct_change'] >= SIGNIFICANT_DROP_THRESHOLD) 
)

df_merged

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,significant_hb_drop,hb_pct_change,timestamp_next,days_to_next,within_7_days,diagnose_db_cs_dbc_specialty_code,diagnose_db_cs_dbc_specialty_code_description,diagnose_db_cs_dbc_diagnosis_code,diagnose_db_cs_dbc_diagnosis_code_description,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,diagnosis_window_start,diagnosis_window_end,next_test_time
19,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.3,9.3,mmol/L,2024-06-17 11:40:00,False,1.063830,2024-06-17 11:41:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-02-27,2024-06-25,2023-11-29,2024-07-02,2024-06-17 11:41:00
32,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,9.4,9.4,mmol/L,2024-06-17 11:41:00,False,NaN,2024-08-13 13:36:00,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-02-27,2024-06-25,2023-11-29,2024-07-02,NaT
44,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.4,8.4,mmol/L,2024-08-13 13:36:00,False,NaN,2024-10-25 13:06:00,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-06-26,2024-10-23,2024-03-28,2024-10-30,NaT
56,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,8.0,8.0,mmol/L,2024-10-25 13:06:00,False,-1.265823,2024-10-25 13:07:00,0.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-10-24,2025-02-20,2024-07-26,2025-02-27,2024-10-25 13:07:00
69,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,Hemoglobine,7.9,7.9,mmol/L,2024-10-25 13:07:00,True,14.130435,2025-01-29 11:07:00,95.0,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2024-10-24,2025-02-20,2024-07-26,2025-02-27,2025-01-29 11:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.8,5.8,mmol/L,2020-06-15 09:25:00,False,-1.754386,2020-06-17 11:50:00,2.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19,2020-06-17 11:50:00
5094,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.7,5.7,mmol/L,2020-06-17 11:50:00,False,-1.785714,2020-06-19 11:00:00,1.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19,2020-06-19 11:00:00
5098,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,5.6,5.6,mmol/L,2020-06-19 11:00:00,True,13.846154,2020-06-22 08:00:00,2.0,True,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19,2020-06-22 08:00:00
5102,FAA79717FF2C725767E9469350ACECF640E5FCBC,Hemoglobine,6.5,6.5,mmol/L,2020-06-22 08:00:00,False,NaN,NaT,NaN,False,318,"Gastro-enterologie (MDL, maag-darm-lever-arts)",203,Bloedverlies tractus digestivus e.c.i.,2020-06-15,2020-09-12,2020-03-17,2020-09-19,NaT


In [50]:
# Filter only significant drops
significant_drops = df_merged[df_merged['significant_hb_drop']]

# Optional: one row per patient + diagnosis period
patients_with_drop = significant_drops[group_cols].drop_duplicates()
len(patients_with_drop)

hb_patient_lst_diagnosis = significant_drops['pseudo_id'].unique().tolist()
print(len(hb_patient_lst_diagnosis)) # 42
hb_patient_lst_diagnosis

36


['088C9FD98B8B2CBCB597C17C07AC1845B21F0849',
 '0E93D98E82D272810A15FDE70270EE41E9C6DB71',
 '12D65DA1F4CFAC101DE53C050C9037D97F42FC18',
 '15CF926899FB0141DABA60251D292FCC89C94C1B',
 '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682',
 '29B3653AE690547AB14AC7FCD32B21A561D5FF9A',
 '2E7E9399C8366C94770E35BC822203C1B0BDAB07',
 '355F07B9B6AF154431F346CBC4A6722294250C43',
 '3605432FE03B28514E3927DA7E5C53BD177BFD31',
 '395219FA6E57FE6DA93F36F3EB68F4667D325895',
 '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE',
 '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C',
 '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B',
 '708E355575D017D3349A8FADE56EC4CE1059C187',
 '7561B5E32DE738229F2D658FBAACEFFE80B50600',
 '895342BFD0D1C32FFF8F009C0E20CBE97B04677D',
 '922EF6916F338885CD4EA526758009EF4B9562FF',
 '94876C5A2D72759210A854698D5A5E012C2ED946',
 '96764B9E95688ACB64A6C8E99AC2E6D5828BED12',
 '9763CE55BAB3BF537B9309F9A7A6085F86ED878A',
 '9A5F28B3534569F4F2EF8F9C7CCD12F102A511D3',
 'A75FEC66C73E482282B5537091BFBD0814458EAA',
 'A9A2482C

In [51]:
df_hb_filtered[df_hb_filtered['pseudo_id'] == 'BEBEFC726C577A7B40F1A467F4E59746FBC7F76B']

for i in range(len(df_hb_filtered)):
    if (df_hb_filtered['pseudo_id'].iloc[i] == 'BEBEFC726C577A7B40F1A467F4E59746FBC7F76B') & (df_hb_filtered['lab_measurement_start_date'].iloc[i].month == 3) & (df_hb_filtered['lab_measurement_start_date'].iloc[i].year == 2021):
        print(df_hb_filtered.iloc[i])


IAT Testing

In [52]:
iat_df = df_hb[df_hb['lab_measurement_description'].str.lower().str.contains('irr|irregulaire', na=False)]
iat_df


,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date
445,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Irregulaire antistoffen,NEG.,NaN,NaN,2021-09-30 09:30:00
731,12D65DA1F4CFAC101DE53C050C9037D97F42FC18,Irregulaire antistoffen,NEG.,NaN,NaN,2022-07-19 11:50:00
774,12D65DA1F4CFAC101DE53C050C9037D97F42FC18,Irregulaire antistoffen,NEG.,NaN,NaN,2022-07-04 08:00:00
885,15CF926899FB0141DABA60251D292FCC89C94C1B,Irregulaire antistoffen,NEG.,NaN,NaN,2024-03-20 15:32:00
1457,23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682,Irregulaire antistoffen,NEG.,NaN,NaN,2018-12-02 08:00:00
...,...,...,...,...,...,...
9913,F63683C1B401C1929E4B45EDBCE3E4CE7757EE9F,Irregulaire antistoffen,NEG.,NaN,NaN,2025-02-17 09:37:00
9990,F797097BA39F81B05EEC3F251973743CA03CB9B4,Irregulaire antistoffen,NEG.,NaN,NaN,2024-12-25 14:30:00
10027,F797097BA39F81B05EEC3F251973743CA03CB9B4,Irregulaire antistoffen,NEG.,NaN,NaN,2024-12-22 20:22:00
10131,F8C241E69061BB450778B7A4CC336FF520469A0A,Irregulaire antistoffen,NEG.,NaN,NaN,2023-03-24 08:00:00


In [53]:
iat_df['lab_measurement_test_value_string'].value_counts()

NEG.     72
Pos.      2
TEKST     2
Name: lab_measurement_test_value_string, dtype: int64

In [54]:
iat_df[iat_df['lab_measurement_test_value_string'] != 'NEG.']
# iat_df['lab_measurement_test_value_string                                         '].iloc[0]

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date
8162,D25F6785232C8650378EAE7E1D994F3506AEF6DA,Irregulaire antistoffen,Pos.,NaN,NaN,2022-09-09 08:00:00
8163,D25F6785232C8650378EAE7E1D994F3506AEF6DA,Irr.Antist.,TEKST,NaN,NaN,2022-09-09 08:00:00
8234,D25F6785232C8650378EAE7E1D994F3506AEF6DA,Irregulaire antistoffen,Pos.,NaN,NaN,2022-09-05 08:00:00
8235,D25F6785232C8650378EAE7E1D994F3506AEF6DA,Irr.Antist.,TEKST,NaN,NaN,2022-09-05 08:00:00


In [55]:
iat_patiënten = iat_df['pseudo_id'].unique()
len(iat_patiënten) #46  
iat_patiënten_lst = iat_patiënten.tolist()
iat_patiënten

array(['0DD73490997F61870C32B3BB7C9CDE4E801FFF7D',
       '12D65DA1F4CFAC101DE53C050C9037D97F42FC18',
       '15CF926899FB0141DABA60251D292FCC89C94C1B',
       '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682',
       '29B3653AE690547AB14AC7FCD32B21A561D5FF9A',
       '2E7E9399C8366C94770E35BC822203C1B0BDAB07',
       '355F07B9B6AF154431F346CBC4A6722294250C43',
       '3605432FE03B28514E3927DA7E5C53BD177BFD31',
       '37BFA228DFA04BE0596DCCC63D245B42B3A15727',
       '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE',
       '40FF2EE78841C453C1C673E9047BABC03DD6CCEE',
       '4D22E67480A038F4FEA67046F5706560CBB06C27',
       '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C',
       '5C16934C56472DEB342641D17FB13F9AEFEEFFAB',
       '618F149E27BFA64EBFEC903D49A75AAF88BD75B2',
       '65F35D3FB57B75161F6854ACD38FBEB77A0394B0',
       '6CF1734B040468B6802E26DFA5BC6B6FF6869362',
       '706C18D256797B3BAFC980C49CA839BB467F6B5F',
       '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B',
       '7561B5E32DE738229F2D658

In [56]:
df_alleen_iat_patiënten = df_hb[df_hb['pseudo_id'].isin(iat_patiënten)]
df_alleen_iat_patiënten

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date
400,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Natrium,138,138.0,mmol/L,2021-10-11 12:30:00
401,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Kalium,2.0,2.0,mmol/L,2021-10-11 12:30:00
402,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Ureum,<0.8,NaN,mmol/L,2021-10-11 12:30:00
403,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Kreatinine,51,51.0,µmol/L,2021-10-11 12:30:00
404,0DD73490997F61870C32B3BB7C9CDE4E801FFF7D,Opm. dialyse,Edial,NaN,NaN,2021-10-11 12:30:00
...,...,...,...,...,...,...
10328,FAA79717FF2C725767E9469350ACECF640E5FCBC,MCV,78,78.0,fL,2020-06-15 09:25:00
10329,FAA79717FF2C725767E9469350ACECF640E5FCBC,Natrium,124,124.0,mmol/L,2020-06-15 09:25:00
10330,FAA79717FF2C725767E9469350ACECF640E5FCBC,Kalium,4.2,4.2,mmol/L,2020-06-15 09:25:00
10331,FAA79717FF2C725767E9469350ACECF640E5FCBC,Ureum,9.9,9.9,mmol/L,2020-06-15 09:25:00


In [57]:
patients_in_both = []
for i in range(len(hb_patient_lst_diagnosis)):
    for j in range(len(iat_patiënten)):
        # print('lst:', agib_patient_lst[j])
        if hb_patient_lst_diagnosis[i] == iat_patiënten[j]:
            # print('Patient is both:', df_reports_scopie_patient['pseudo_id'][i])
            if hb_patient_lst_diagnosis[i] not in patients_in_both:
                patients_in_both.append(hb_patient_lst_diagnosis[i])
                
len(patients_in_both)

27

In [58]:
len(hb_patient_lst_diagnosis)

36

In [59]:
bora_metrics = set(hb_patient_lst_diagnosis) | set(iat_patiënten_lst)
len(bora_metrics)

55

In [60]:
relevant_tests = [
    'Hemoglobine',
    'Hematocriet',
    'INR',
    'APTT',
    'Protrombinetijd',
    'Ureum',
    'MCV',
    'Trombocyten'
]

df_tests_filtered = df_hb[df_hb['lab_measurement_description'].isin(relevant_tests)]
df_tests_filtered

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.9,6.90,mmol/L,2024-07-22 09:29:00
5,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,MCV,97,97.00,fL,2024-07-22 09:29:00
6,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Trombocyten,379,379.00,x10^9/L,2024-07-22 09:29:00
16,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.2,6.20,mmol/L,2024-07-15 08:38:00
17,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,MCV,96,96.00,fL,2024-07-15 08:38:00
...,...,...,...,...,...,...
10417,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.3,7.30,mmol/L,2018-10-17 11:11:00
10418,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hematocriet,0.36,0.36,L/L,2018-10-17 11:11:00
10419,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,MCV,87,87.00,fL,2018-10-17 11:11:00
10420,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Trombocyten,261,261.00,x10^9/L,2018-10-17 11:11:00


In [61]:
df_tests_filtered['Hb'] = None
df_tests_filtered['Hematocriet'] = None
df_tests_filtered['MCV'] = None
df_tests_filtered['INR'] = None
df_tests_filtered['APTT'] = None
df_tests_filtered['Protrombinetijd'] = None
df_tests_filtered['Ureum'] = None
df_tests_filtered['Trombocyten'] = None

for i in range(len(df_tests_filtered)):
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Hemoglobine':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 7.5:
            df_tests_filtered['Hb'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 7.5:
            df_tests_filtered['Hb'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Hematocriet':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 0.35:
            df_tests_filtered['Hematocriet'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 0.35:
            df_tests_filtered['Hematocriet'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'MCV':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 80:
            df_tests_filtered['MCV'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 80:
            df_tests_filtered['MCV'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'INR':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 1.5:
            df_tests_filtered['INR'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 1.5:
            df_tests_filtered['INR'].iloc[i] = 'high'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'APTT':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 35:
            df_tests_filtered['APTT'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 35:
            df_tests_filtered['APTT'].iloc[i] = 'high'
    
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Protrombinetijd':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 15:
            df_tests_filtered['Protrombinetijd'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 15:
            df_tests_filtered['Protrombinetijd'].iloc[i] = 'high'
    
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Ureum':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 7.5:
            df_tests_filtered['Ureum'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 7.5:
            df_tests_filtered['Ureum'].iloc[i] = 'high'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Trombocyten':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 150:
            df_tests_filtered['Trombocyten'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 150:
            df_tests_filtered['Trombocyten'].iloc[i] = 'normal'


    
df_tests_filtered


C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tests_filtered['Hb'] = None
C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tests_filtered['Hematocriet'] = None
C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,pseudo_id,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,Hb,Hematocriet,MCV,INR,APTT,Protrombinetijd,Ureum,Trombocyten
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.9,6.90,mmol/L,2024-07-22 09:29:00,low,None,None,None,None,None,None,None
5,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,MCV,97,97.00,fL,2024-07-22 09:29:00,None,None,normal,None,None,None,None,None
6,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Trombocyten,379,379.00,x10^9/L,2024-07-22 09:29:00,None,None,None,None,None,None,None,normal
16,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,Hemoglobine,6.2,6.20,mmol/L,2024-07-15 08:38:00,low,None,None,None,None,None,None,None
17,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,MCV,96,96.00,fL,2024-07-15 08:38:00,None,None,normal,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10417,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hemoglobine,7.3,7.30,mmol/L,2018-10-17 11:11:00,low,None,None,None,None,None,None,None
10418,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Hematocriet,0.36,0.36,L/L,2018-10-17 11:11:00,None,normal,None,None,None,None,None,None
10419,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,MCV,87,87.00,fL,2018-10-17 11:11:00,None,None,normal,None,None,None,None,None
10420,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,Trombocyten,261,261.00,x10^9/L,2018-10-17 11:11:00,None,None,None,None,None,None,None,normal


In [62]:
df_tests_wide = df_tests_filtered.pivot_table(
    index=['pseudo_id', 'lab_measurement_start_date'],
    columns='lab_measurement_description',
    values='lab_measurement_test_value_numeric',
    aggfunc='first'
).reset_index()

df_tests_wide


lab_measurement_description,pseudo_id,lab_measurement_start_date,APTT,Hematocriet,Hemoglobine,INR,MCV,Protrombinetijd,Trombocyten,Ureum
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-03 08:00:00,NaN,NaN,NaN,1.8,NaN,NaN,NaN,NaN
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-05 08:01:00,NaN,NaN,NaN,3.4,NaN,NaN,356.0,NaN
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-08 08:00:00,NaN,NaN,NaN,1.2,NaN,NaN,NaN,NaN
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-11 08:00:00,NaN,NaN,NaN,2.3,NaN,NaN,NaN,NaN
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-15 08:38:00,NaN,NaN,6.2,NaN,96.0,NaN,402.0,5.8
...,...,...,...,...,...,...,...,...,...,...
801,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-03 08:00:00,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN
802,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-05 08:00:00,NaN,NaN,NaN,1.6,NaN,NaN,NaN,NaN
803,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-08 08:00:00,NaN,NaN,NaN,4.1,NaN,NaN,NaN,NaN
804,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-09 12:25:00,NaN,NaN,8.7,2.1,88.0,NaN,NaN,NaN


In [63]:
df_tests_wide['low_hb'] = df_tests_wide['Hemoglobine'] < 7.5
df_tests_wide['low_hematocriet'] = df_tests_wide['Hematocriet'] < 0.35
df_tests_wide['low_mcv'] = df_tests_wide['MCV'] < 80
df_tests_wide['high_inr'] = df_tests_wide['INR'] > 1.5
df_tests_wide['high_aptt'] = df_tests_wide['APTT'] > 35
df_tests_wide['high_pt'] = df_tests_wide['Protrombinetijd'] > 15
df_tests_wide['high_urea'] = df_tests_wide['Ureum'] > 7.5
df_tests_wide['low_plt'] = df_tests_wide['Trombocyten'] < 150

df_tests_wide

lab_measurement_description,pseudo_id,lab_measurement_start_date,APTT,Hematocriet,Hemoglobine,INR,MCV,Protrombinetijd,Trombocyten,Ureum,low_hb,low_hematocriet,low_mcv,high_inr,high_aptt,high_pt,high_urea,low_plt
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-03 08:00:00,NaN,NaN,NaN,1.8,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-05 08:01:00,NaN,NaN,NaN,3.4,NaN,NaN,356.0,NaN,False,False,False,True,False,False,False,False
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-08 08:00:00,NaN,NaN,NaN,1.2,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-11 08:00:00,NaN,NaN,NaN,2.3,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-15 08:38:00,NaN,NaN,6.2,NaN,96.0,NaN,402.0,5.8,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-03 08:00:00,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False
802,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-05 08:00:00,NaN,NaN,NaN,1.6,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False
803,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-08 08:00:00,NaN,NaN,NaN,4.1,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False
804,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-09 12:25:00,NaN,NaN,8.7,2.1,88.0,NaN,NaN,NaN,False,False,False,True,False,False,False,False


In [64]:
# Combine features: 1) Hb + Urea  OR  2 or more coagulopathy markers
df_tests_wide['likely_GIB'] = (
    (df_tests_wide['low_hb'] & df_tests_wide['high_urea']) |
    (df_tests_wide[['low_hb', 'high_inr', 'low_plt', 'high_aptt', 'high_pt']].sum(axis=1) >= 2)
)

df_tests_wide.value_counts('likely_GIB')
df_tests_wide


lab_measurement_description,pseudo_id,lab_measurement_start_date,APTT,Hematocriet,Hemoglobine,INR,MCV,Protrombinetijd,Trombocyten,Ureum,low_hb,low_hematocriet,low_mcv,high_inr,high_aptt,high_pt,high_urea,low_plt,likely_GIB
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-03 08:00:00,NaN,NaN,NaN,1.8,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-05 08:01:00,NaN,NaN,NaN,3.4,NaN,NaN,356.0,NaN,False,False,False,True,False,False,False,False,False
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-08 08:00:00,NaN,NaN,NaN,1.2,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-11 08:00:00,NaN,NaN,NaN,2.3,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2024-07-15 08:38:00,NaN,NaN,6.2,NaN,96.0,NaN,402.0,5.8,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-03 08:00:00,NaN,NaN,NaN,5.6,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False
802,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-05 08:00:00,NaN,NaN,NaN,1.6,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False
803,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-08 08:00:00,NaN,NaN,NaN,4.1,NaN,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False
804,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2021-03-09 12:25:00,NaN,NaN,8.7,2.1,88.0,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False


In [65]:
df_scopie = pd.read_csv('a:/bloeding-met-patientenlijst-gedetailleerd/df_scopie.csv')
df_regex = pd.read_csv('a:/bloeding-met-patientenlijst-gedetailleerd/df_regex.csv')

# # Ensure date columns are in datetime format
df_scopie['verslagen_report_start_date'] = pd.to_datetime(df_scopie['verslagen_report_start_date'])
df_regex['verslagen_report_start_date'] = pd.to_datetime(df_regex['verslagen_report_start_date'])

# Merge both DataFrames on pseudo_id and report date
df_combined = pd.merge(
    df_regex,
    df_scopie,	
    on=['pseudo_id', 'verslagen_report_start_date', 'verslagen_report_content'],
    how='inner'  # or 'inner' if you only want overlapping rows
)

len(df_combined)
df_combined.head()

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,is_seh,is_scopie,scopie_after_seh
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",False,True,True,True,2020-11-24,2021-01-07,False,False,False
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...,False,True,True,True,2020-11-24,2021-01-07,False,False,False
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,False,True,True,True,2020-11-24,2021-01-07,False,True,False
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,False,True,True,True,2020-11-24,2021-01-07,False,True,False
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...,False,True,True,True,2020-11-24,2021-01-07,False,True,False


In [66]:
df_combined = df_combined.drop_duplicates(subset=['pseudo_id', 'verslagen_report_start_date', 'verslagen_report_content'])

len(df_combined)

11047

In [67]:
print(len(df_scopie), len(df_regex))
print(len(df_scopie.drop_duplicates(subset=['pseudo_id', 'verslagen_report_start_date', 'verslagen_report_content', 'is_seh', 'is_scopie', 'scopie_after_seh'])))
print(len(df_regex.drop_duplicates(subset=['pseudo_id', 'verslagen_report_start_date', 'verslagen_report_content', 'agib_final'])))

print(df_scopie['verslagen_report_start_date'].dtype)
print(df_regex['verslagen_report_start_date'].dtype)



11090 11090
11047
11047
datetime64[ns]
datetime64[ns]


In [68]:
df_combined['verslagen_report_start_date'] = pd.to_datetime(df_combined['verslagen_report_start_date'])
df_hb['lab_measurement_start_date'] = pd.to_datetime(df_hb['lab_measurement_start_date'])

df_combined['report_date'] = df_combined['verslagen_report_start_date'].dt.date
df_hb['lab_date'] = df_hb['lab_measurement_start_date'].dt.date


In [69]:
df_merged = pd.merge(
    df_combined,
    df_hb.assign(lab_date=df_hb['lab_measurement_start_date'].dt.date),
    left_on=['pseudo_id', 'report_date'],
    right_on=['pseudo_id', 'lab_date'],
    how='left'  # <-- keeps all rows from df_combined
)

df_merged


,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,is_seh,is_scopie,scopie_after_seh,report_date,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,lab_date
0,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 15:06:00,"Maag-, Darm- en Leverziekten","Dhr. A.J. Dingemans, huisarts\r\n[STREETNAME] ...",False,True,True,True,2020-11-24,2021-01-07,False,False,False,2020-11-26,NaN,NaN,NaN,NaN,NaT,NaN
1,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-26 09:53:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies obv diver...,False,True,True,True,2020-11-24,2021-01-07,False,False,False,2020-11-26,NaN,NaN,NaN,NaN,NaT,NaN
2,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 14:13:00,"Maag-, Darm- en Leverziekten",COLOSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] [L...,False,True,True,True,2020-11-24,2021-01-07,False,True,False,2020-11-25,NaN,NaN,NaN,NaN,NaT,NaN
3,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 13:48:00,"Maag-, Darm- en Leverziekten",GASTROSCOPIE\r\n\r\nBetreft\r\nMw. [INITIALS] ...,False,True,True,True,2020-11-24,2021-01-07,False,True,False,2020-11-25,NaN,NaN,NaN,NaN,NaT,NaN
4,046D1FFEBDD40E1665D0ABA6DD8FC9F8BC4351C6,2020-11-25 08:47:00,"Maag-, Darm- en Leverziekten",Samenvatting: \nRectaal bloedverlies ; eenmali...,False,True,True,True,2020-11-24,2021-01-07,False,True,False,2020-11-25,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2015-03-20 08:13:00,Interne Geneeskunde,Samenvatting: \n1e consult\r\n-Type 1e consult...,False,False,True,False,NaN,NaN,False,False,False,2015-03-20,NaN,NaN,NaN,NaN,NaT,NaN
19307,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2015-01-14 15:39:00,Interne Geneeskunde,Samenvatting: \nDecursus\r\n-Type decursus: De...,False,False,False,False,NaN,NaN,False,False,False,2015-01-14,NaN,NaN,NaN,NaN,NaT,NaN
19308,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2014-12-21 09:31:00,Spoedeisende Hulp,Samenvatting: \nVerpleegkundige verslaglegging...,False,False,False,False,NaN,NaN,True,False,False,2014-12-21,NaN,NaN,NaN,NaN,NaT,NaN
19309,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2010-11-10 21:03:00,Spoedeisende Hulp,Samenvatting: \nMedisch Dossier\r\n[ Vk Sputov...,False,False,False,False,NaN,NaN,True,False,False,2010-11-10,NaN,NaN,NaN,NaN,NaT,NaN


In [70]:
relevant_tests = [
    'Hemoglobine',
    'Hematocriet',
    'INR',
    'APTT',
    'Protrombinetijd',
    'Ureum',
    'MCV',
    'Trombocyten'
]

df_merged_tests_filtered = df_merged[df_merged['lab_measurement_description'].isin(relevant_tests)]
df_merged_tests_filtered 

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,is_seh,is_scopie,scopie_after_seh,report_date,lab_measurement_description,lab_measurement_test_value_string,lab_measurement_test_value_numeric,lab_measurement_test_value_unit,lab_measurement_start_date,lab_date
54,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,True,False,NaN,2025-01-30,Hemoglobine,8.3,8.30,mmol/L,2025-01-30 15:32:00,2025-01-30
55,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,True,False,NaN,2025-01-30,Hematocriet,0.41,0.41,L/L,2025-01-30 15:32:00,2025-01-30
56,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,True,False,NaN,2025-01-30,MCV,89,89.00,fL,2025-01-30 15:32:00,2025-01-30
59,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,True,False,NaN,2025-01-30,Ureum,11.0,11.00,mmol/L,2025-01-30 15:32:00,2025-01-30
65,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 15:22:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nTrauma capitis...,False,True,False,True,2024-10-24,2025-02-20,True,False,NaN,2025-01-30,Hemoglobine,8.3,8.30,mmol/L,2025-01-30 15:32:00,2025-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19220,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,True,False,False,2018-10-17,Hemoglobine,7.3,7.30,mmol/L,2018-10-17 11:11:00,2018-10-17
19221,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,True,False,False,2018-10-17,Hematocriet,0.36,0.36,L/L,2018-10-17 11:11:00,2018-10-17
19222,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,True,False,False,2018-10-17,MCV,87,87.00,fL,2018-10-17 11:11:00,2018-10-17
19223,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,True,False,False,2018-10-17,Trombocyten,261,261.00,x10^9/L,2018-10-17 11:11:00,2018-10-17


In [71]:
df_tests_filtered = df_merged_tests_filtered.copy()

In [72]:
df_tests_filtered['Hb'] = None
df_tests_filtered['Hematocriet'] = None
df_tests_filtered['MCV'] = None
df_tests_filtered['INR'] = None
df_tests_filtered['APTT'] = None
df_tests_filtered['Protrombinetijd'] = None
df_tests_filtered['Ureum'] = None
df_tests_filtered['Trombocyten'] = None

for i in range(len(df_tests_filtered)):
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Hemoglobine':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 7.5:
            df_tests_filtered['Hb'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 7.5:
            df_tests_filtered['Hb'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Hematocriet':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 0.35:
            df_tests_filtered['Hematocriet'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 0.35:
            df_tests_filtered['Hematocriet'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'MCV':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 80:
            df_tests_filtered['MCV'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 80:
            df_tests_filtered['MCV'].iloc[i] = 'normal'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'INR':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 1.5:
            df_tests_filtered['INR'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 1.5:
            df_tests_filtered['INR'].iloc[i] = 'high'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'APTT':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 35:
            df_tests_filtered['APTT'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 35:
            df_tests_filtered['APTT'].iloc[i] = 'high'
    
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Protrombinetijd':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 15:
            df_tests_filtered['Protrombinetijd'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 15:
            df_tests_filtered['Protrombinetijd'].iloc[i] = 'high'
    
    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Ureum':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 7.5:
            df_tests_filtered['Ureum'].iloc[i] = 'normal'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 7.5:
            df_tests_filtered['Ureum'].iloc[i] = 'high'

    if df_tests_filtered['lab_measurement_description'].iloc[i] == 'Trombocyten':
        if df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] < 150:
            df_tests_filtered['Trombocyten'].iloc[i] = 'low'
        elif df_tests_filtered['lab_measurement_test_value_numeric'].iloc[i] > 150:
            df_tests_filtered['Trombocyten'].iloc[i] = 'normal'


    
df_tests_filtered


C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tests_filtered['Hb'].iloc[i] = 'normal'
C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tests_filtered['Hematocriet'].iloc[i] = 'normal'
C:\Users\Y.vanMegen\AppData\Local\Temp\ipykernel_13868\2654359534.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

,pseudo_id,verslagen_report_start_date,verslagen_report_specialism,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,diagnose_db_cs_dbc_start_date,diagnose_db_cs_dbc_end_date,...,lab_measurement_start_date,lab_date,Hb,Hematocriet,MCV,INR,APTT,Protrombinetijd,Ureum,Trombocyten
54,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,...,2025-01-30 15:32:00,2025-01-30,normal,None,None,None,None,None,None,None
55,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,...,2025-01-30 15:32:00,2025-01-30,None,normal,None,None,None,None,None,None
56,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,...,2025-01-30 15:32:00,2025-01-30,None,None,normal,None,None,None,None,None
59,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 17:05:00,Spoedeisende Hulp,Huisartsen Achtse Barrier\r\nT.a.v. mevr. C.M....,False,True,False,True,2024-10-24,2025-02-20,...,2025-01-30 15:32:00,2025-01-30,None,None,None,None,None,None,high,None
65,088C9FD98B8B2CBCB597C17C07AC1845B21F0849,2025-01-30 15:22:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nTrauma capitis...,False,True,False,True,2024-10-24,2025-02-20,...,2025-01-30 15:32:00,2025-01-30,normal,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19220,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,...,2018-10-17 11:11:00,2018-10-17,low,None,None,None,None,None,None,None
19221,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,...,2018-10-17 11:11:00,2018-10-17,None,normal,None,None,None,None,None,None
19222,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,...,2018-10-17 11:11:00,2018-10-17,None,None,normal,None,None,None,None,None
19223,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 12:35:00,Spoedeisende Hulp,Reden van komst / Verwijzing: \nReden van koms...,False,True,False,True,2018-10-17,2019-01-14,...,2018-10-17 11:11:00,2018-10-17,None,None,None,None,None,None,None,normal


In [73]:
df_tests_wide = df_merged_tests_filtered.pivot_table(
    index=['pseudo_id', 'lab_measurement_start_date', 'verslagen_report_content', 'agib_final', 'gib_final', 'scopie_final', 'treatment_final', 'scopie_after_seh'],
    columns='lab_measurement_description',	#	is_seh	is_scopie	scopie_after_seh	report_date	lab_measurement_description	lab_measurement_test_value_string	lab_measurement_test_value_numeric	lab_measurement_test_value_unit	lab_measurement_start_date	
    values='lab_measurement_test_value_numeric',
    aggfunc='first'
).reset_index()

df_tests_wide


lab_measurement_description,pseudo_id,lab_measurement_start_date,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,scopie_after_seh,APTT,Hematocriet,Hemoglobine,INR,MCV,Protrombinetijd,Trombocyten,Ureum
0,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,0.28,5.9,NaN,92.0,NaN,305.0,NaN
1,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,0.28,5.9,NaN,92.0,NaN,305.0,NaN
2,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,True,False,False,NaN,0.28,5.9,NaN,92.0,NaN,305.0,NaN
3,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,NaN,5.5,NaN,92.0,NaN,NaN,NaN
4,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,NaN,5.5,NaN,92.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
472,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
473,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-25 08:00:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,8.2,NaN,98.0,NaN,205.0,NaN
474,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 11:11:00,Aan de weledelgeleerde heer\r\ndrs. F.J.G.M. V...,False,True,False,True,False,NaN,0.36,7.3,NaN,87.0,NaN,261.0,10.4


In [74]:
df_tests_wide['low_hb'] = df_tests_wide['Hemoglobine'] < 7.5
df_tests_wide['low_hematocriet'] = df_tests_wide['Hematocriet'] < 0.35
df_tests_wide['low_mcv'] = df_tests_wide['MCV'] < 80
df_tests_wide['high_inr'] = df_tests_wide['INR'] > 1.5
df_tests_wide['high_aptt'] = df_tests_wide['APTT'] > 35
df_tests_wide['high_pt'] = df_tests_wide['Protrombinetijd'] > 15
df_tests_wide['high_urea'] = df_tests_wide['Ureum'] > 7.5
df_tests_wide['low_plt'] = df_tests_wide['Trombocyten'] < 150

df_tests_wide

lab_measurement_description,pseudo_id,lab_measurement_start_date,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,scopie_after_seh,APTT,Hematocriet,...,Trombocyten,Ureum,low_hb,low_hematocriet,low_mcv,high_inr,high_aptt,high_pt,high_urea,low_plt
0,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,0.28,...,305.0,NaN,True,True,False,False,False,False,False,False
1,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,0.28,...,305.0,NaN,True,True,False,False,False,False,False,False
2,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,True,False,False,NaN,0.28,...,305.0,NaN,True,True,False,False,False,False,False,False
3,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,NaN,...,NaN,NaN,True,False,False,False,False,False,False,False
4,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,NaN,...,NaN,NaN,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
472,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
473,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-25 08:00:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,205.0,NaN,False,False,False,False,False,False,False,False
474,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 11:11:00,Aan de weledelgeleerde heer\r\ndrs. F.J.G.M. V...,False,True,False,True,False,NaN,0.36,...,261.0,10.4,True,False,False,False,False,False,True,False


In [75]:
# Combine features: 1) Hb + Urea  OR  2 or more coagulopathy markers
df_tests_wide['likely_GIB'] = (
    df_tests_wide['gib_final'] |
    (df_tests_wide['low_hb'] & df_tests_wide['high_urea']) |
    (df_tests_wide[['low_hb', 'high_inr', 'low_plt', 'high_aptt', 'high_pt']].sum(axis=1) >= 2)
)

df_tests_wide.value_counts('likely_GIB')
df_tests_wide


lab_measurement_description,pseudo_id,lab_measurement_start_date,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,scopie_after_seh,APTT,Hematocriet,...,Ureum,low_hb,low_hematocriet,low_mcv,high_inr,high_aptt,high_pt,high_urea,low_plt,likely_GIB
0,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,0.28,...,NaN,True,True,False,False,False,False,False,False,True
1,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,0.28,...,NaN,True,True,False,False,False,False,False,False,True
2,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,True,False,False,NaN,0.28,...,NaN,True,True,False,False,False,False,False,False,True
3,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,NaN,...,NaN,True,False,False,False,False,False,False,False,True
4,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,NaN,...,NaN,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,NaN,False,False,False,False,False,False,False,False,True
472,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,NaN,False,False,False,False,False,False,False,False,True
473,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-25 08:00:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,NaN,False,False,False,False,False,False,False,False,True
474,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 11:11:00,Aan de weledelgeleerde heer\r\ndrs. F.J.G.M. V...,False,True,False,True,False,NaN,0.36,...,10.4,True,False,False,False,False,False,True,False,True


In [76]:
print(df_tests_wide['likely_GIB'].value_counts())
print(df_tests_wide[df_tests_wide['likely_GIB'] == True]['pseudo_id'].unique())
print(len(df_tests_wide[df_tests_wide['likely_GIB'] == True]['pseudo_id'].unique()))


True     370
False    106
Name: likely_GIB, dtype: int64
['0E93D98E82D272810A15FDE70270EE41E9C6DB71'
 '15CF926899FB0141DABA60251D292FCC89C94C1B'
 '20AAD8E38E7C9D75E44F9EA52336B7003ED239BD'
 '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682'
 '29B3653AE690547AB14AC7FCD32B21A561D5FF9A'
 '395219FA6E57FE6DA93F36F3EB68F4667D325895'
 '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE'
 '40FF2EE78841C453C1C673E9047BABC03DD6CCEE'
 '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C'
 '6CF1734B040468B6802E26DFA5BC6B6FF6869362'
 '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B'
 '7561B5E32DE738229F2D658FBAACEFFE80B50600'
 '895342BFD0D1C32FFF8F009C0E20CBE97B04677D'
 '96764B9E95688ACB64A6C8E99AC2E6D5828BED12'
 '9763CE55BAB3BF537B9309F9A7A6085F86ED878A'
 '9A5F28B3534569F4F2EF8F9C7CCD12F102A511D3'
 'A8FA2D94EA62234A97F2D21737D6046198E67B20'
 'B4F185DF5CC6BA66AC14EBD0972CCCF7AFC77ED5'
 'BAE82DBED446134D3759C32B5457D88A06F8F6AD'
 'BEBEFC726C577A7B40F1A467F4E59746FBC7F76B'
 'C775682FCB29A890D7B64B211F17DFF3D8867951'
 'C7C86E1AA957D3F18

In [77]:
# Start with GIB-positive patients
df_tests_wide['likely_AGIB'] = df_tests_wide['gib_final']

df = df_tests_wide.copy()


In [78]:
df['likely_AGIB'] = (
    df['gib_final'] &
    (
        df['scopie_after_seh'] |                 # urgent scoped
        df['high_urea'] |                        # upper source suggested by urea
        df['agib_final']                         # not negated and matched AGIB
        # df['gib_match'].str.contains("melena|hematemesis", case=False, na=False)
    )
)

df

lab_measurement_description,pseudo_id,lab_measurement_start_date,verslagen_report_content,agib_final,gib_final,scopie_final,treatment_final,scopie_after_seh,APTT,Hematocriet,...,low_hb,low_hematocriet,low_mcv,high_inr,high_aptt,high_pt,high_urea,low_plt,likely_GIB,likely_AGIB
0,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,0.28,...,True,True,False,False,False,False,False,False,True,True
1,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,0.28,...,True,True,False,False,False,False,False,False,True,False
2,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 11:50:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,True,False,False,NaN,0.28,...,True,True,False,False,False,False,False,False,True,False
3,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Reden van komst / Verwijzing: \nReden van koms...,False,True,True,False,True,NaN,NaN,...,True,False,False,False,False,False,False,False,True,True
4,0E93D98E82D272810A15FDE70270EE41E9C6DB71,2019-11-24 20:00:00,Samenvatting: \nBloedverlies per anum onder AS...,False,True,False,False,False,NaN,NaN,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
472,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-23 17:23:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
473,F8D116F01EE0039678998F393FA337C10AD4F4E2,2020-09-25 08:00:00,Samenvatting: \nVoorgeschiedenis: \nJicht\nHyp...,False,True,False,True,False,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
474,FD8C682C1F4FDA1E5EC0B760D30875556419BD71,2018-10-17 11:11:00,Aan de weledelgeleerde heer\r\ndrs. F.J.G.M. V...,False,True,False,True,False,NaN,0.36,...,True,False,False,False,False,False,True,False,True,True


In [79]:
print(df_tests_wide['likely_AGIB'].value_counts())
print(df_tests_wide[df_tests_wide['likely_AGIB'] == True]['pseudo_id'].unique())
print(len(df_tests_wide[df_tests_wide['likely_AGIB'] == True]['pseudo_id'].unique()))

True     336
False    140
Name: likely_AGIB, dtype: int64
['0E93D98E82D272810A15FDE70270EE41E9C6DB71'
 '15CF926899FB0141DABA60251D292FCC89C94C1B'
 '20AAD8E38E7C9D75E44F9EA52336B7003ED239BD'
 '23E295F4F8F7550C76B6C22ABDA7DFFB3FCF1682'
 '29B3653AE690547AB14AC7FCD32B21A561D5FF9A'
 '395219FA6E57FE6DA93F36F3EB68F4667D325895'
 '3B21EF377C1BA327A67C2C951A6CB78BEAD5B3FE'
 '40FF2EE78841C453C1C673E9047BABC03DD6CCEE'
 '52EE37A2B81CD38A3062B4B7B216EDBB8BE07E6C'
 '6CF1734B040468B6802E26DFA5BC6B6FF6869362'
 '7087EB7CD49F67A7A8E51203DE7C0F9FFBBFB51B'
 '7561B5E32DE738229F2D658FBAACEFFE80B50600'
 '895342BFD0D1C32FFF8F009C0E20CBE97B04677D'
 '96764B9E95688ACB64A6C8E99AC2E6D5828BED12'
 '9763CE55BAB3BF537B9309F9A7A6085F86ED878A'
 '9A5F28B3534569F4F2EF8F9C7CCD12F102A511D3'
 'A8FA2D94EA62234A97F2D21737D6046198E67B20'
 'BAE82DBED446134D3759C32B5457D88A06F8F6AD'
 'BEBEFC726C577A7B40F1A467F4E59746FBC7F76B'
 'C775682FCB29A890D7B64B211F17DFF3D8867951'
 'C7C86E1AA957D3F18CD7803ABCBE9D63C07AFCCA'
 'CB30ED9CF2F8BE2F